In [1]:
import pandas as pd
import numpy as np
import gc
from collections import Counter
import csv
import math
import gzip
import pathlib

from nltk.tokenize.casual import TweetTokenizer

In [2]:
def parse(path, max_rows=100000):
    g = gzip.open(path, 'rb')
    for i, l in enumerate(g):
        if i == max_rows:
            return eval(l)
        yield eval(l)

def get_df(path, max_rows=100000):
    i = 0
    df = {}
    for d in parse(path, max_rows):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

cwd = pathlib.Path.cwd()
repo_dir = cwd.parent
dataset_dir = repo_dir / "datasets" / "amazon_health"

df = get_df(dataset_dir / 'reviews_Health_and_Personal_Care_5.json.gz', max_rows=300000)

In [3]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,ALC5GH8CAMAI7,159985130X,AnnN,"[1, 1]",This is a great little gadget to have around. ...,5.0,Handy little gadget,1294185600,"01 5, 2011"
1,AHKSURW85PJUE,159985130X,"AZ buyer ""AZ buyer""","[1, 1]",I would recommend this for a travel magnifier ...,4.0,Small & may need to encourage battery,1329523200,"02 18, 2012"
2,A38RMU1Y5TDP9,159985130X,"Bob Tobias ""Robert Tobias""","[75, 77]",What I liked was the quality of the lens and t...,4.0,Very good but not great,1275955200,"06 8, 2010"
3,A1XZUG7DFXXOS4,159985130X,Cat lover,"[56, 60]",Love the Great point light pocket magnifier! ...,4.0,great addition to your purse,1202428800,"02 8, 2008"
4,A1MS3M7M7AM13X,159985130X,Cricketoes,"[1, 1]",This is very nice. You pull out on the magnifi...,5.0,Very nice and convenient.,1313452800,"08 16, 2011"


In [4]:
def my_round(x, base=.05, prec=2):
    return round(base * round(float(x) / base), prec)


def split_dataset_into_even_class_distributions(X_data, Y_data, mini_batch_size=32):
    """
    Split a dataset into train, validate, and test sets that have the same distribution of classes as the original data.
    Also this function ensures the resulting datasets are multiples of the mini_batch_size

    :param X_data: Numpy 2d array or pandas dataframe, each row is a record
    :param Y_data: Pandas Series, the categorial class labels
    :param train_size: float 0-1, quantity of the original dataset to be in the train set (val is created from the residual)
    :param test_size: float 0-1, quantity of the original dataset to be in the test set (val is created from the residual)
    :param mini_batch_size: Int, the size of the mini batches to ensure each dataset is a multiple of that values

    :return: 6 data sets of X and Y
    """

    dist = Counter(val for val in Y_data)
    print('Total class distribution:', dict(dist))

    Y_data.reset_index(drop=True, inplace=True)
    X_data.reset_index(drop=True, inplace=True)

    train_ix = []
    validate_ix = []
    test_ix = []
    for cls in dist.keys():
        num_train = my_round(dist[cls] * train_size, base=mini_batch_size)
        num_test = my_round(dist[cls] * test_size, base=mini_batch_size)

        cls_targets = Y_data[Y_data == cls]

        full_ixs = np.random.choice(cls_targets.index, size=num_train, replace=False)

        train_ix += list(full_ixs)

        cls_targets = cls_targets[~cls_targets.index.isin(full_ixs)]

        full_ixs = np.random.choice(cls_targets.index, size=num_test, replace=False)

        test_ix += list(full_ixs)

        cls_targets = cls_targets[~cls_targets.index.isin(full_ixs)]

        validate_ix += list(cls_targets.index)

    X_train = X_data[train_ix]
    Y_train = Y_data[train_ix].reset_index(drop=True)

    X_validate = X_data[validate_ix]
    Y_validate = Y_data[validate_ix].reset_index(drop=True)

    X_test = X_data[test_ix]
    Y_test = Y_data[test_ix].reset_index(drop=True)

    print('Train class distribution:', dict(Counter(val for val in Y_train)))
    print('Validate class distribution:', dict(Counter(val for val in Y_validate)))
    print('Test class distribution:', dict(Counter(val for val in Y_test)))

    return X_train, Y_train, X_validate, Y_validate, X_test, Y_test


def parse_df_into_sets_by_tertiary_grouping(df, ter="ID", y="Score", x="text", train_prop=.7, test_prop=.2):

    ideal_train_size = math.floor(len(df)*train_prop)
    ideal_test_size = math.floor(len(df)*test_prop)
    ideal_val_size = len(df) - (ideal_train_size + ideal_test_size)

    title_avg_count = df[ter].value_counts().mean()

    titles = df[ter].unique().tolist()

    train_titles = []
    train_count = 0

    while train_count <= ideal_train_size - title_avg_count:
        title = np.random.choice(titles)
        titles.remove(title)
        train_titles.append(title)
        train_count += len(df[df[ter] == title])

    test_titles = []
    test_count = 0

    while test_count <= ideal_test_size - title_avg_count:
        title = np.random.choice(titles)
        titles.remove(title)
        test_titles.append(title)
        test_count += len(df[df[ter] == title]) 

    val_titles = titles
    val_count = len(df[df[ter].isin(val_titles)])

    print("Ideal train size is:", ideal_train_size, "whearas the actual is:", train_count)
    print("Ideal val size is:", ideal_val_size, "whearas the actual is:", val_count)
    print("Ideal test size is:", ideal_test_size, "whearas the actual is:", test_count)

    df_trn = df[df[ter].isin(train_titles)]
    df_val = df[df[ter].isin(val_titles)]
    df_tst = df[df[ter].isin(test_titles)]
    
    print("Train class distribution:", df_trn[y].value_counts())
    print("Val class distribution:", df_val[y].value_counts())
    print("Test class distribution:", df_tst[y].value_counts())

    return df_trn[x], df_trn[y], df_val[x], df_val[y], df_tst[x], df_tst[y]

def balance_classes(X_data, Y_data, min_class_value=None):
    
    kes = Y_data.value_counts().keys().tolist()
    
    min_class = Y_data.value_counts().keys()[-1]
    
    if not min_class_value:
        min_class_value = Y_data.value_counts().values[-1]
    
    all_ix = []
            
    for ke in kes:
        ke_indexes = Y_data[Y_data == ke].index.tolist()
        if len(ke_indexes) == min_class_value:
            all_ix += ke_indexes
        else:
            all_ix += list(np.random.choice(ke_indexes, size=min_class_value, replace=False))
        
    X_data = X_data[all_ix]
    Y_data = Y_data[all_ix]
    
    print("Y class distribution:", Y_data.value_counts())
    
    return X_data, Y_data

In [5]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,ALC5GH8CAMAI7,159985130X,AnnN,"[1, 1]",This is a great little gadget to have around. ...,5.0,Handy little gadget,1294185600,"01 5, 2011"
1,AHKSURW85PJUE,159985130X,"AZ buyer ""AZ buyer""","[1, 1]",I would recommend this for a travel magnifier ...,4.0,Small & may need to encourage battery,1329523200,"02 18, 2012"
2,A38RMU1Y5TDP9,159985130X,"Bob Tobias ""Robert Tobias""","[75, 77]",What I liked was the quality of the lens and t...,4.0,Very good but not great,1275955200,"06 8, 2010"
3,A1XZUG7DFXXOS4,159985130X,Cat lover,"[56, 60]",Love the Great point light pocket magnifier! ...,4.0,great addition to your purse,1202428800,"02 8, 2008"
4,A1MS3M7M7AM13X,159985130X,Cricketoes,"[1, 1]",This is very nice. You pull out on the magnifi...,5.0,Very nice and convenient.,1313452800,"08 16, 2011"


In [7]:
# Unbalanced

X_train, Y_train, X_validate, Y_validate, X_test, Y_test = parse_df_into_sets_by_tertiary_grouping(df=df,
                                                                                                   ter="asin",
                                                                                                   y="overall",
                                                                                                   x="reviewText",
                                                                                                   train_prop=.4, 
                                                                                                   test_prop=.4)

N_CLASSES = len(Y_train.value_counts())
print(N_CLASSES, "classes")

df_trn = pd.DataFrame(np.array([Y_train, X_train]).T, columns=['label', 'text'])
df_val = pd.DataFrame(np.array([Y_validate, X_validate]).T, columns=['label', 'text'])
df_tst = pd.DataFrame(np.array([Y_test, X_test]).T, columns=['label', 'text'])

if len(df_val) > 25000:
    df_val = df_val.sample(n=25000)

df_tst = df_tst.sample(n=25000)

for size in [100, 300, 1000]:
    df_trn.sample(n=size*N_CLASSES).to_csv(dataset_dir / ("fair_unbalanced_"+str(size)) / "train.csv", index=False, quoting=csv.QUOTE_ALL, encoding='utf-8')
    df_val.to_csv(dataset_dir / ("fair_unbalanced_"+str(size)) / "validate.csv", index=False, quoting=csv.QUOTE_ALL, encoding='utf-8')
    df_tst.to_csv(dataset_dir / ("fair_unbalanced_"+str(size)) / "test.csv", index=False, quoting=csv.QUOTE_ALL, encoding='utf-8')

Ideal train size is: 120000 whearas the actual is: 119994
Ideal val size is: 60000 whearas the actual is: 60024
Ideal test size is: 120000 whearas the actual is: 119982
Train class distribution: 5.0    72978
4.0    23205
3.0    11869
2.0     6033
1.0     5909
Name: overall, dtype: int64
Val class distribution: 5.0    36628
4.0    11768
3.0     5657
1.0     3034
2.0     2937
Name: overall, dtype: int64
Test class distribution: 5.0    73299
4.0    23285
3.0    11484
1.0     5983
2.0     5931
Name: overall, dtype: int64
5 classes


In [8]:
# Balanced
X_train, Y_train, X_validate, Y_validate, X_test, Y_test = parse_df_into_sets_by_tertiary_grouping(df=df,
                                                                                                   ter="asin",
                                                                                                   y="overall",
                                                                                                   x="reviewText",
                                                                                                   train_prop=.4, 
                                                                                                   test_prop=.4)

X_validate, Y_validate = balance_classes(X_data=X_validate, Y_data=Y_validate)
X_test, Y_test = balance_classes(X_data=X_test, Y_data=Y_test, min_class_value=5000)

df_val = pd.DataFrame(np.array([Y_validate, X_validate]).T, columns=['label', 'text'])
df_tst = pd.DataFrame(np.array([Y_test, X_test]).T, columns=['label', 'text'])

for size in [100, 300, 1000]:
    X_train_crnt, Y_train_crnt = balance_classes(X_data=X_train, Y_data=Y_train, min_class_value=size)
    df_trn = pd.DataFrame(np.array([Y_train_crnt, X_train_crnt]).T, columns=['label', 'text'])
    df_trn.to_csv(dataset_dir / ("fair_balanced_"+str(size)) / "train.csv", index=False, quoting=csv.QUOTE_ALL, encoding='utf-8')
    df_val.to_csv(dataset_dir / ("fair_balanced_"+str(size)) / "validate.csv", index=False, quoting=csv.QUOTE_ALL, encoding='utf-8')
    df_tst.to_csv(dataset_dir / ("fair_balanced_"+str(size)) / "test.csv", index=False, quoting=csv.QUOTE_ALL, encoding='utf-8')

Ideal train size is: 120000 whearas the actual is: 119982
Ideal val size is: 60000 whearas the actual is: 60033
Ideal test size is: 120000 whearas the actual is: 119985
Train class distribution: 5.0    72895
4.0    23237
3.0    11758
1.0     6086
2.0     6006
Name: overall, dtype: int64
Val class distribution: 5.0    36836
4.0    11629
3.0     5777
2.0     2896
1.0     2895
Name: overall, dtype: int64
Test class distribution: 5.0    73174
4.0    23392
3.0    11475
2.0     5999
1.0     5945
Name: overall, dtype: int64
Y class distribution: 1.0    2895
2.0    2895
3.0    2895
4.0    2895
5.0    2895
Name: overall, dtype: int64
Y class distribution: 1.0    5000
2.0    5000
3.0    5000
4.0    5000
5.0    5000
Name: overall, dtype: int64
Y class distribution: 2.0    100
1.0    100
3.0    100
4.0    100
5.0    100
Name: overall, dtype: int64
Y class distribution: 2.0    300
1.0    300
3.0    300
4.0    300
5.0    300
Name: overall, dtype: int64
Y class distribution: 2.0    1000
1.0    1000
3

In [ ]:
dataset_dir / ("fair_unbalanced_"+str(size)) / "train.csv"

In [ ]:
df = pd.read_csv(dataset_dir / ("fair_balanced_"+str(100)) / "train.csv")
df['label'].value_counts()